In [43]:
import pandas as pd
import sqlite3

# Load the dataset
file_path = 'top_spotify_73_tori_cleaned.csv'
df = pd.read_csv(file_path)
df.head()

,name,artists,daily_rank,country,snapshot_date,album_name,danceability,energy,acousticness,valence
0,Love Somebody,Morgan Wallen,1,US,2024-10-21,Love Somebody,0.619,0.843,0.00948,0.746
1,Die With A Smile,"Lady Gaga, Bruno Mars",2,US,2024-10-21,Die With A Smile,0.521,0.592,0.30800,0.535
2,Taste,Sabrina Carpenter,3,US,2024-10-21,Short n' Sweet,0.674,0.907,0.10100,0.721
3,BIRDS OF A FEATHER,Billie Eilish,4,US,2024-10-21,HIT ME HARD AND SOFT,0.747,0.507,0.20000,0.438
4,APT.,"ROSÉ, Bruno Mars",5,US,2024-10-21,APT.,0.777,0.783,0.02830,0.939


In [50]:
# Create an SQLite database in memory again
conn = sqlite3.connect(":memory:")

# Store the DataFrame into the SQLite database
df.to_sql("spotify_songs", conn, index=False, if_exists="replace")

# Query for the top 10 songs based on daily rank in the United States on 12/31/2023
query_2023 = """
SELECT name, artists, daily_rank, danceability, energy, valence, acousticness
FROM spotify_songs
WHERE snapshot_date = '2023-10-20'
ORDER BY daily_rank ASC
LIMIT 10
"""
top_10_2023_df = pd.read_sql_query(query_2023, conn)

# Query for the top 10 songs based on daily rank in the United States on 12/31/2023
query_2024 = """
SELECT name, artists, daily_rank, danceability, energy, valence, acousticness
FROM spotify_songs
WHERE snapshot_date = '2024-10-20'
ORDER BY daily_rank ASC
LIMIT 10
"""
top_10_2024_df = pd.read_sql_query(query_2024, conn)


# Close the SQLite connection
conn.close()

# Display the two DataFrames
print(top_10_2023_df)



                                            name                      artists  \
0                             IDGAF (feat. Yeat)                  Drake, Yeat   
1                                         MONACO                    Bad Bunny   
2                          My Love Mine All Mine                       Mitski   
3  I Remember Everything (feat. Kacey Musgraves)  Zach Bryan, Kacey Musgraves   
4                             Paint The Town Red                     Doja Cat   
5           First Person Shooter (feat. J. Cole)               Drake, J. Cole   
6                                   Cruel Summer                 Taylor Swift   
7                                       fukumean                        Gunna   
8                                         greedy                   Tate McRae   
9                                     Last Night                Morgan Wallen   

   daily_rank  danceability  energy  valence  acousticness  
0           1         0.663   0.670    0.138   

In [51]:
print(top_10_2024_df)

                                    name                     artists  \
0                          Love Somebody               Morgan Wallen   
1                       Die With A Smile       Lady Gaga, Bruno Mars   
2                     BIRDS OF A FEATHER               Billie Eilish   
3                                  Taste           Sabrina Carpenter   
4                       Good Luck, Babe!               Chappell Roan   
5          Timeless (with Playboi Carti)   The Weeknd, Playboi Carti   
6                                    Who                       Jimin   
7                            Sailor Song                  Gigi Perez   
8  I Had Some Help (Feat. Morgan Wallen)  Post Malone, Morgan Wallen   
9                               Espresso           Sabrina Carpenter   

   daily_rank  danceability  energy  valence  acousticness  
0           1         0.619   0.843    0.746       0.00948  
1           2         0.521   0.592    0.535       0.30800  
2           3         0.

In [62]:
average_dance_23 = top_10_2023_df['danceability'].mean()
average_enery_23 = top_10_2023_df['energy'].mean()
average_valence_23 = top_10_2023_df['valence'].mean()
average_acousticness_23  = top_10_2023_df['acousticness'].mean()
print("2023 Averages:")
print("Danceability:", average_dance_23,
      "Energy:", average_enery_23,
      "Valence:", average_valence_23,
      "Acousticness:", average_acousticness_23)

average_dance_24 = top_10_2024_df['danceability'].mean()
average_enery_24 = top_10_2024_df['energy'].mean()
average_valence_24 = top_10_2024_df['valence'].mean()
average_acousticness_24  = top_10_2024_df['acousticness'].mean()
print("2024 Averages:")
print("Danceability", average_dance_24,
      "Energy:", average_enery_24,
      "Valence:", average_valence_24,
      "Acousticnes:", average_acousticness_24)

2023 Averages:
Danceability: 0.6357999999999999 Energy: 0.598 Valence: 0.36239999999999994 Acousticness: 0.28601000000000004
2024 Averages:
Danceability 0.6527 Energy: 0.6971999999999999 Valence: 0.6367 Acousticnes: 0.15542700000000004


In [52]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_file, save
from bokeh.plotting import figure, show
from bokeh.transform import dodge

# Get data for Bokeh visualization
source = ColumnDataSource(top_10_2023_df)
track_names = top_10_2023_df["name"]

# Create the bar chart
p = figure(
    x_range=track_names,
    title="US Top 10 Songs as of October 20th 2023 - Danceability, Energy, Acousticness, Valence",
    height=650,
    width=900,
    toolbar_location=None,
    tools=""
)
# Add bars for each metric
p.vbar(x=dodge('name', -0.3, range=p.x_range), top='danceability', source=source,
       width=0.2, color="#392061", legend_label="Danceability")

p.vbar(x=dodge('name',  -.1,  range=p.x_range), top='energy', source=source,
       width=0.2, color="#F758AF", legend_label="Energy")

p.vbar(x=dodge('name',  0.1, range=p.x_range), top='acousticness', source=source,
       width=0.2, color="#42CAFD", legend_label="Acousticness")

p.vbar(x=dodge('name',  0.3, range=p.x_range), top='valence', source=source,
       width=0.2, color="#A4F5AF", legend_label="Valence")

# Add HoverTool
hover_danceability = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Danceability"), ("Value", "@danceability")],
renderers=[p.renderers[0]]
)
hover_energy = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Energy"), ("Value", "@energy")],
    renderers=[p.renderers[1]]
)
hover_acousticness = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Acousticness"), ("Value", "@acousticness")],
    renderers=[p.renderers[2]]
)
hover_valence = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Valence"), ("Value", "@valence")],
    renderers=[p.renderers[3]]
)

# Add HoverTool to the figure
p.add_tools(hover_danceability, hover_energy, hover_acousticness, hover_valence)

# Configure plot
p.y_range.start = 0
p.y_range.end = 1.18
p.xaxis.major_label_orientation = -1.25
p.xaxis.axis_label = "Song Name"
p.yaxis.axis_label = "Value"
p.xgrid.grid_line_color = None
p.legend.title = "Metrics"
p.legend.orientation = "horizontal"
p.legend.location = "top_right"

# Display plot
output_file("top_10_songs_2023.html")
save(p)

'/Users/torishirk/Desktop/Bootcamp/Project-3/top_10_songs_2023.html'

In [53]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_file, save
from bokeh.plotting import figure, show
from bokeh.transform import dodge

# Get data for Bokeh visualization
source = ColumnDataSource(top_10_2024_df)
track_names = top_10_2024_df["name"]

# Create the bar chart
p = figure(
    x_range=track_names,
    title="US Top 10 Songs as of October 20th 2024 - Danceability, Energy, Acousticness, Valence",
    height=650,
    width=900,
    toolbar_location=None,
    tools=""
)
# Add bars for each metric
p.vbar(x=dodge('name', -0.3, range=p.x_range), top='danceability', source=source,
       width=0.2, color="#392061", legend_label="Danceability")

p.vbar(x=dodge('name',  -.1,  range=p.x_range), top='energy', source=source,
       width=0.2, color="#F758AF", legend_label="Energy")

p.vbar(x=dodge('name',  0.1, range=p.x_range), top='acousticness', source=source,
       width=0.2, color="#42CAFD", legend_label="Acousticness")

p.vbar(x=dodge('name',  0.3, range=p.x_range), top='valence', source=source,
       width=0.2, color="#A4F5AF", legend_label="Valence")

# Add HoverTool
hover_danceability = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Danceability"), ("Value", "@danceability")],
renderers=[p.renderers[0]]
)
hover_energy = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Energy"), ("Value", "@energy")],
    renderers=[p.renderers[1]]
)
hover_acousticness = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Acousticness"), ("Value", "@acousticness")],
    renderers=[p.renderers[2]]
)
hover_valence = HoverTool(
    tooltips=[("Song", "@name"), ("Artist", "@artists"), ("Metric", "Valence"), ("Value", "@valence")],
    renderers=[p.renderers[3]]
)

# Add HoverTool to the figure
p.add_tools(hover_danceability, hover_energy, hover_acousticness, hover_valence)

# Configure plot
p.y_range.start = 0
p.y_range.end = 1.18
p.xaxis.major_label_orientation = -1.25
p.xaxis.axis_label = "Song Name"
p.yaxis.axis_label = "Value"
p.xgrid.grid_line_color = None
p.legend.title = "Metrics"
p.legend.orientation = "horizontal"
p.legend.location = "top_right"

# Display plot
output_file("top_10_songs_2024.html")
save(p)

'/Users/torishirk/Desktop/Bootcamp/Project-3/top_10_songs_2024.html'